Se construirá un data set de los modismos con las columnas, palabra, significado y ejemplo

In [1]:
import pandas as pd
import re

In [2]:
path_file_academia = "AcademiaColombianaLengua/diccionario_colombianismos.txt"

data = []
current_entry = {}

with open(path_file_academia, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue

        if line.startswith("**PALABRA:**"):
            if 'palabra' in current_entry:
                # Append previous entry if a new one starts
                data.append(current_entry)
            current_entry = {"palabra": line.replace("**PALABRA:**", "").strip()}
        elif line.startswith("**SIGNIFICADO:**"):
            current_entry["significado"] = line.replace("**SIGNIFICADO:**", "").strip()
        elif line.startswith("**EJEMPLO:**"):
            current_entry["ejemplo"] = line.replace("**EJEMPLO:**", "").strip()

# Add the last entry
if current_entry:
    data.append(current_entry)

df = pd.DataFrame(data)

# Display the first few rows of the dataframe
df.head()

,palabra,significado,ejemplo
0,abajeño,Propio o nativo de las costas o de las tierras...,Empezaron cultivos de tabaco y a elaborar un d...
1,abalear,Disparar a alguien o a algo de manera repetida...,El funcionario resultó ileso a pesar de que el...
2,abalear,Herir o matar a alguien con disparos de un arm...,Limpiaba la hojarasca cuando hombres armados l...
3,abaleo,Situación en la que hay disparos repetidos que...,Por la calle doce se oía un tremendo abaleo y ...
4,abanicar,"En el beisbol, fallar el bateador al no tocar ...",El bateador se ponchó abanicando el tercer lan...


In [3]:
df.shape

(7800, 3)

In [9]:
df[df["significado"] == "Definición no encontrada."]

,palabra,significado,ejemplo
13,abombao,Definición no encontrada.,Ejemplo no encontrado.
17,abozado,Definición no encontrada.,Ejemplo no encontrado.
21,abrigo,Definición no encontrada.,Ejemplo no encontrado.
76,achote,Definición no encontrada.,Ejemplo no encontrado.
79,achucharrar,Definición no encontrada.,Ejemplo no encontrado.
...,...,...,...
7678,yin,Definición no encontrada.,Ejemplo no encontrado.
7704,zacatín,Definición no encontrada.,Ejemplo no encontrado.
7740,zoco,Definición no encontrada.,Ejemplo no encontrado.
7759,zábila,Definición no encontrada.,Ejemplo no encontrado.


Al haber duplicado tantos modismos por sus multiples definiciones, algunos quedaron vacios. Luego de revisar manualmente que no fueran falsos positivos, se eliminaron del dataset

In [7]:
df_limpio = df[
    (df["significado"] != "Definición no encontrada.")
].copy()

In [8]:
df_limpio[df_limpio["significado"] == "Definición no encontrada."]

,palabra,significado,ejemplo


Con este data set tenemos 7800 modismos

In [12]:
path_file_CaroCuervo = "Lexicc - CaroCuervo\\Diccionario_breve_de_Colombiaismos_slown_text.txt"

data_caro = []
# Regex to find the word, definition and example
# It handles multiple definitions in the same line
pattern = re.compile(r"-\s\*\*(?P<palabra>[^|*]+)(?:\|\|.*?)?\*\*.*?(?=\d+\.|\s[A-ZÁ-ÚÑ])(?P<definicion>.*?)(?:\s\*(?P<ejemplo>.*?)\*(?:\s\(.*?\))?\s*$|$)")


with open(path_file_CaroCuervo, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line.startswith("- **"):
            continue
        
        # Split the line by numbered definitions like "2.", "3."
        parts = re.split(r'\s\*\*\d+\.\*\*\s', line)
        
        base_palabra_match = re.match(r'-\s\*\*(.*?)\*\*', parts[0])
        if not base_palabra_match:
            continue
        
        base_palabra = base_palabra_match.group(1).split('||')[0].strip()

        for i, part in enumerate(parts):
            content = part
            if i == 0:
                # For the first part, remove the initial "- **palabra**"
                content = re.sub(r'-\s\*\*.*?\*\*\s*', '', part)

            # Clean up content
            # Remove grammatical tags like 'tr.', 'm.', 'adj.'
            content = re.sub(r'\b(tr|intr|prnl|m|f|adj|adv|interj|sust|coloq|despect|obsol|pop|rur|vulg)\.\s', '', content)
            # Remove regional tags like '*Boy., Cund.*'
            content = re.sub(r'\*([A-Z][a-z]+(?:\.\s[A-Z][a-z]+)*\.)\*', '', content)
            content = content.strip()

            # Separate definition and example
            ejemplo_match = re.search(r'\s\*(.*?)\*.*$', content)
            
            definicion = content
            ejemplo = None

            if ejemplo_match:
                ejemplo = ejemplo_match.group(1).strip()
                definicion = content[:ejemplo_match.start()].strip()

            if definicion:
                data_caro.append({
                    "palabra": base_palabra,
                    "significado": definicion,
                    "ejemplo": ejemplo
                })


df_caro = pd.DataFrame(data_caro)
df_caro.head()

,palabra,significado,ejemplo
0,abagó,"*Boy., Cund.* Selección de los mejores frutos ...",Si yo supiera lo cierto/Cuál jue el que te aco...
1,abalear,Disparar balas sobre alguien o algo.,None
2,abalear,Herir o matar a balazos.,None
3,abaleo,Tiroteo.,En el abaleo resultaron heridas tres personas.
4,abanico,*Costa Atl.* Ventilador eléctrico.,Prende el abanico que hace mucho calor


In [13]:
df_caro.shape

(1852, 3)

Con este encontramos 1852 modismos, ahora se procederá a mezclar ambos datasets.

Antes de eso realizaremos un analisis de completitud de ambos datasets

In [14]:
print("Análisis de completitud para el DataFrame df:")
print(df.isnull().sum())

print("\nAnálisis de completitud para el DataFrame df_caro:")
print(df_caro.isnull().sum())

Análisis de completitud para el DataFrame df:
palabra        0
significado    0
ejemplo        0
dtype: int64

Análisis de completitud para el DataFrame df_caro:
palabra           0
significado       0
ejemplo        1234
dtype: int64


Podemos ver que el primer dataset esta completo y el segundo de CaroCuervo le faltan la mayoria de ejemplos. Vamos a mezclar ambos y volveremos a realizar este análisis de completitud.

Es encesario agregar la fuente de donde se sacaron cada grupo de datos. Para esrto agregaremos una columna fuente donde se dira que es de la fuente Academia Colombiana o Diccionario CaroCuervo

In [23]:
df["fuente"] = "AcademiaColombianaLengua"
df_caro["fuente"] = "DiccionarioCaroCuervo"

In [24]:
display(df.head())
display(df_caro.head())

,palabra,significado,ejemplo,fuente
0,abajeño,Propio o nativo de las costas o de las tierras...,Empezaron cultivos de tabaco y a elaborar un d...,AcademiaColombianaLengua
1,abalear,Disparar a alguien o a algo de manera repetida...,El funcionario resultó ileso a pesar de que el...,AcademiaColombianaLengua
2,abaleo,Situación en la que hay disparos repetidos que...,Por la calle doce se oía un tremendo abaleo y ...,AcademiaColombianaLengua
3,abanicar,"En el beisbol, fallar el bateador al no tocar ...",El bateador se ponchó abanicando el tercer lan...,AcademiaColombianaLengua
4,abanico,Aparato eléctrico que crea una corriente de ai...,No podía dormir la siesta de la tarde sin el a...,AcademiaColombianaLengua


,palabra,significado,ejemplo,fuente
0,abagó,"*Boy., Cund.* Selección de los mejores frutos ...",Si yo supiera lo cierto/Cuál jue el que te aco...,DiccionarioCaroCuervo
1,abalear,Disparar balas sobre alguien o algo.,None,DiccionarioCaroCuervo
2,abalear,Herir o matar a balazos.,None,DiccionarioCaroCuervo
3,abaleo,Tiroteo.,En el abaleo resultaron heridas tres personas.,DiccionarioCaroCuervo
4,abanico,*Costa Atl.* Ventilador eléctrico.,Prende el abanico que hace mucho calor,DiccionarioCaroCuervo


In [25]:
df_merged = pd.concat([df, df_caro], ignore_index=True)

In [26]:
print("Forma del dataframe mezclado:", df_merged.shape)
display(df_merged.head())

print("\nAnálisis de completitud para el DataFrame mezclado:")
print(df_merged.isnull().sum())

Forma del dataframe mezclado: (7685, 4)


,palabra,significado,ejemplo,fuente
0,abajeño,Propio o nativo de las costas o de las tierras...,Empezaron cultivos de tabaco y a elaborar un d...,AcademiaColombianaLengua
1,abalear,Disparar a alguien o a algo de manera repetida...,El funcionario resultó ileso a pesar de que el...,AcademiaColombianaLengua
2,abaleo,Situación en la que hay disparos repetidos que...,Por la calle doce se oía un tremendo abaleo y ...,AcademiaColombianaLengua
3,abanicar,"En el beisbol, fallar el bateador al no tocar ...",El bateador se ponchó abanicando el tercer lan...,AcademiaColombianaLengua
4,abanico,Aparato eléctrico que crea una corriente de ai...,No podía dormir la siesta de la tarde sin el a...,AcademiaColombianaLengua



Análisis de completitud para el DataFrame mezclado:
palabra           0
significado       0
ejemplo        1234
fuente            0
dtype: int64


In [31]:
df_merged.to_csv("modismos_Dataset.csv", sep=";", index=False)

Con esto obtenemos los datos finales para empezar la investigación